In [46]:
# install pulp, if you already have it, you can skip this step
# ! pip install pulp

In [47]:
import numpy as np
from pulp import *
import pulp
import random
import matplotlib.pyplot as plt

# ResNet18

In [48]:
class args():
    def __init__(self):
        pass
args = args
args.model_size_limit = 0.5 # the model size limit, 0.25 means, 4bit_model_size * 0.75 + 8bit_model_size * 0.25
args.bops_limit = 0.5 # same definition as above
args.latency_limit = 0.5 # same definition as above

In [49]:
# those are different matrics

# Hutchinson_trace means the trace of Hessian for each weight matrix.
# Particular, it has 19 elements since ResNet18 has 18 layers, removing the first and last layer 
# we still have 16 layers (the other three come from residual connection layer)
# These values are already normlized, i.e., Trace / # arameters
Hutchinson_trace = np.array([0.06857826, 0.03162379, 0.03298575, 0.01205663, 0.02222431, 0.00596336, 0.06931772, 0.00807129, 0.00372905, 0.00530698, 0.00209011, 0.00737569, 0.00210454, 0.00151197, 0.00158041,0.00078146, 0.00451841, 0.00098745, 0.00072944])
# Delta Weight 8 bit Square means \| W_fp32 - W_int8  \|_2^2
delta_weights_8bit_square = np.array([0.0235, 0.0125, 0.0102, 0.0082, 0.0145, 0.0344, 0.0023, 0.0287, 0.0148, 0.0333, 0.0682, 0.0027, 0.0448, 0.0336, 0.0576, 0.1130, 0.0102, 0.0947, 0.0532]) #  = (w_fp32 - w_int8)^2
# Delta Weight 4 bit Square means \| W_fp32 - W_int4  \|_2^2
delta_weights_4bit_square = np.array([6.7430, 3.9691, 3.3281, 2.6796, 4.7277, 10.5966, 0.6827, 9.0942, 4.8857, 10.7599, 21.7546, 0.8603, 14.5324, 10.9651, 18.7706, 36.4044, 3.1572, 29.6994, 17.4016]) #  = (w_fp32 - w_int4)^2
# here is the sensitivity different between 4 and 8
sensitivity_difference_between_4_8 = Hutchinson_trace * ( delta_weights_8bit_square  - delta_weights_4bit_square ) 
# number of paramers of each layer
parameters = np.array([ 36864, 36864, 36864, 36864, 73728, 147456, 8192, 147456, 147456, 294912, 589824, 32768, 589824, 589824, 1179648, 2359296, 131072, 2359296, 2359296]) / 1024 / 1024 # make it millition based (1024 is due to Byte to MB see next cell for model size computation)
# Bit Operators of each layer
bops = np.array([115605504, 115605504, 115605504, 115605504, 57802752, 115605504, 6422528, 115605504, 115605504, 57802752, 115605504, 6422528, 115605504, 115605504, 57802752, 115605504, 6422528, 115605504, 115605504]) / 1000000 # num of bops for each layer/block
# latency for INT4 and INT8, measured on T4 GPU
latency_int4 = np.array([0.21094404, 0.21092674, 0.21104113, 0.21086851, 0.13642465, 0.19167506, 0.02532183, 0.19148203, 0.19142914, 0.11395316, 0.20556011, 0.01917474, 0.20566918, 0.20566509, 0.13185102, 0.22287786, 0.01790088, 0.22304611, 0.22286099])
latency_int8 = np.array([0.36189111, 0.36211718, 0.31141909, 0.30454471, 0.19184896, 0.38948934, 0.0334169, 0.38904905, 0.3892859, 0.19134735, 0.34307431, 0.02802354, 0.34313329, 0.34310756, 0.21117103, 0.37376585, 0.02896843, 0.37398187, 0.37405185])
latency_difference_between_4_8 = latency_int8 - latency_int4

In [50]:
# do some calculatations first
# model size
model_size_32bit = np.sum(parameters) * 4. # MB
model_size_8bit = model_size_32bit / 4. # 8bit model is 1/4 of 32bit model 
model_size_4bit = model_size_32bit / 8. # 4bit model is 1/8 of 32bit model 
# as mentioned previous, that's how we set the model size limit
model_size_limit = model_size_4bit + (model_size_8bit - model_size_4bit) * args.model_size_limit

# bops
bops_8bit = bops / 4. / 4. # For Wx, we have two matrices, so that we need the (bops / 4 / 4)
bops_4bit = bops / 8. / 8. # Similar to above line
bops_difference_between_4_8 = bops_8bit - bops_4bit
bops_limit = np.sum(bops_4bit) + (np.sum(bops_8bit) - np.sum(bops_4bit)) * args.bops_limit # similar to model size

# latency 
latency_limit = np.sum(latency_int4) + (np.sum(latency_int8) - np.sum(latency_int4)) * args.latency_limit # similar to model size

print('model_size_limit: ', model_size_limit)
print('bops_limit: ', bops_limit)
print('latency_limit: ', latency_limit)

model_size_limit:  7.98046875
bops_limit:  66.23232000000002
latency_limit:  4.246179795


In [29]:
# 初始化粒子群
def initialize_particles(num_particles, num_variables):
    particles = []
    velocities = []
    for _ in range(num_particles):
        particle = [random.choice([1, 2]) for _ in range(num_variables)]
        particles.append(particle)
    for _ in range(num_particles):
        velocity = [random.choice([1, 2]) for _ in range(num_variables)]
        velocities.append(velocity)
    return particles, velocities


# 粒子群算法主函数
def PSO(num_particles=30, max_iterations=100, w=0.5, c1=1.5, c2=1.5):
    num_variables = len(parameters)
    particles, velocities = initialize_particles(num_particles, num_variables)
    global_best_particle = particles[0]
    global_best_fitness = fitness_func(global_best_particle)
    local_best_particles = particles[:]  # 初始化每个粒子的局部最优位置为当前位置
    local_best_fitness = [fitness_func(p) for p in particles]  # 记录每个粒子的局部最优适应度
    best_Model_Size_History = []
    best_fitness_History = []

    for iter in range(max_iterations):
        for i, particle in enumerate(particles):
            fitness = fitness_func(particle)
            if fitness < local_best_fitness[i]:  # 更新局部最优
                local_best_particles[i] = particle
                local_best_fitness[i] = fitness
            if fitness < global_best_fitness:  # 更新全局最优
                global_best_fitness = fitness
                global_best_particle = particle
            # 自适应变异
            if np.random.rand() > 0.8:
                index = np.random.randint(0, len(particle))
                if np.random.uniform(1, 2) < 1.5:
                    particle[index] = 1
                else:
                    particle[index] = 2
        for i in range(num_particles):
            particles[i], velocities[i] = update_particle(particles[i], velocities[i], global_best_particle, local_best_particles[i], w, c1, c2)
        global_best_particle = np.array(global_best_particle)
        best_fitness_History.append(global_best_fitness)
        best_Model_Size_History.append(np.sum(global_best_particle * parameters * 4 * 4 / 32))
        # print('{}: {}'.format(iter, global_best_particle))
        
    return global_best_particle, global_best_fitness, best_Model_Size_History, best_fitness_History


# 更新粒子的速度和位置
def update_particle(particle, velocity, global_best_particle, local_best_particle, w, c1, c2):
    new_particle = particle[:]
    new_velocity = velocity[:]
    for i in range(len(particle)):
        r1, r2 = random.random(), random.random()
        # 更新速度
        new_velocity[i] = w * velocity[i] + c1 * r1 * (local_best_particle[i] - particle[i]) + c2 * r2 * (global_best_particle[i] - particle[i])
        # 更新位置
        new_particle[i] = 1 if (new_particle[i] + velocity[i]) < 1.5 else 2

        # new_particle[i] = particle[i] + w * (new_particle[i] - particle[i]) + c1 * r1 * (global_best_particle[i] - particle[i]) + c2 * r2 * (particle[i] - particle[i])
        # new_particle[i] = 1 if new_particle[i] < 1.5 else 2  # 根据速度更新位置，小于1.5取1，大于等于1.5取2

    return new_particle, new_velocity

### Model Size Constrait

In [32]:
# 计算适应度值
def fitness_func(particle):
    if (particle[4] != particle[6]) or (particle[9] != particle[11]) or (particle[14] != particle[15]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum(0.5 * particle[i] * parameters[i] for i in range(len(particle)))
        if total <= model_size_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大

# 测试
while(True):
    best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
    model_size = np.sum(best_particle * parameters * 4 * 4 / 32)
    result_4 = (best_particle == 1)
    result_8 = (best_particle == 2)
    latency = np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8])
    if model_size < 7.2656 and latency < 4.891:  # best_fitness < -1.2332 and model_size < 7.2656:
        print("最优粒子：", best_particle)
        print("最优适应度值：", best_fitness)
        print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
        result_4 = (best_particle == 1)
        result_8 = (best_particle == 2)
        print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
        print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))
        break


最优粒子： [2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 1 1]
最优适应度值： -1.2328661475630003
Model Size: 6.703125
Bops:  87.00518400000003
Latency:  4.8113527


In [9]:
'''# 画图
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(best_Model_Size_History ) + 1), best_Model_Size_History, marker='o', markersize=0)
plt.xlabel('Iterations')
plt.ylabel('Global Best Fitness')
plt.title('Model Size')
plt.subplot(1, 2, 2)
plt.plot(range(1, len(best_fitness_History ) + 1), best_fitness_History, marker='o', markersize=0)
plt.xlabel('Iterations')
plt.ylabel('Global Best Fitness')
plt.title('fitness')
plt.show()'''

"# 画图\nplt.figure(figsize=(12, 6))\nplt.subplot(1, 2, 1)\nplt.plot(range(1, len(best_Model_Size_History ) + 1), best_Model_Size_History, marker='o', markersize=0)\nplt.xlabel('Iterations')\nplt.ylabel('Global Best Fitness')\nplt.title('Model Size')\nplt.subplot(1, 2, 2)\nplt.plot(range(1, len(best_fitness_History ) + 1), best_fitness_History, marker='o', markersize=0)\nplt.xlabel('Iterations')\nplt.ylabel('Global Best Fitness')\nplt.title('fitness')\nplt.show()"

### Bops Constrait

In [10]:
# 计算适应度值
def fitness_func(particle):
    if (particle[4] != particle[6]) or (particle[9] != particle[11]) or (particle[14] != particle[16]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum([ bops_4bit[i] + (particle[i] - 1) * bops_difference_between_4_8[i] for i in range(len(particle)) ]) 
        if total <= bops_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大


# 测试
best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
print("最优粒子：", best_particle)
print("最优适应度值：", best_fitness)
print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
result_4 = (best_particle == 1)
result_8 = (best_particle == 2)
print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))


最优粒子： [2 2 2 1 2 1 2 2 1 2 2 2 1 1 2 1 2 1 1]
最优适应度值： -1.0728312531000002
Model Size: 6.544921875
Bops:  62.61964800000001
Latency:  4.12641889


# latency Constrait

In [66]:
# 计算适应度值
def fitness_func(particle):
    if (particle[4] != particle[6]) or (particle[9] != particle[11]) or (particle[14] != particle[16]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum([ latency_int4[i] + (particle[i] - 1) * latency_difference_between_4_8[i] for i in range(len(particle)) ])  # if 4 bit, x - 1 = 0, we use bops_4, if 8 bit, x-1=2, we use bops_diff + bos_4 = bops_8
        if total <= latency_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大


# 测试
while(True):
    best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
    result_4 = (best_particle == 1)
    result_8 = (best_particle == 2)
    Latency = np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8])
    if Latency < 4.22 and best_fitness < -1.1:
        print("最优粒子：", best_particle)
        print("最优适应度值：", best_fitness)
        print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
        result_4 = (best_particle == 1)
        result_8 = (best_particle == 2)
        print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
        print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))
        break


最优粒子： [2 1 2 2 2 1 2 2 1 2 2 2 1 1 2 1 2 1 1]
最优适应度值： -1.1050393344820002
Model Size: 6.544921875
Bops:  62.61964800000001
Latency:  4.06890465


In [63]:
# 计算适应度值
def fitness_func(particle):
    if (particle[4] != particle[6]) or (particle[9] != particle[11]) or (particle[14] != particle[16]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum([ latency_int4[i] + (particle[i] - 1) * latency_difference_between_4_8[i] for i in range(len(particle)) ])  # if 4 bit, x - 1 = 0, we use bops_4, if 8 bit, x-1=2, we use bops_diff + bos_4 = bops_8
        if total <= latency_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大


# 测试
while(True):
    best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
    result_4 = (best_particle == 1)
    result_8 = (best_particle == 2)
    Latency = np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8])
    if Latency < 4.22:
        print("最优粒子：", best_particle)
        print("最优适应度值：", best_fitness)
        print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
        result_4 = (best_particle == 1)
        result_8 = (best_particle == 2)
        print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
        print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))
        break


最优粒子： [2 2 2 2 2 2 2 2 1 2 1 2 1 1 1 1 1 1 1]
最优适应度值： -1.0789049253700003
Model Size: 5.7265625
Bops:  65.028096
Latency:  4.19000761


In [22]:
# 计算适应度值
def fitness_func(particle):
    if (particle[4] != particle[6]) or (particle[9] != particle[11]) or (particle[14] != particle[16]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total1 = sum(0.5 * particle[i] * parameters[i] for i in range(len(particle)))

        total2 = sum([ bops_4bit[i] + (particle[i] - 1) * bops_difference_between_4_8[i] for i in range(len(particle)) ]) 

        total3 = sum([ latency_int4[i] + (particle[i] - 1) * latency_difference_between_4_8[i] for i in range(len(particle)) ])  # if 4 bit, x - 1 = 0, we use bops_4, if 8 bit, x-1=2, we use bops_diff + bos_4 = bops_8
        if (total1 <= model_size_limit) and (total2 <= bops_limit) and (total3 <= latency_limit):
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大


# 测试
best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
print("最优粒子：", best_particle)
print("最优适应度值：", best_fitness)
print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
result_4 = (best_particle == 1)
result_8 = (best_particle == 2)
print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))


最优粒子： [2 2 2 2 2 2 2 2 1 2 1 2 1 1 1 1 1 1 1]
最优适应度值： -1.0789049253700003
Model Size: 5.7265625
Bops:  65.028096
Latency:  4.19000761


# ResNet50

In [67]:
class args():
    def __init__(self):
        pass
args = args
args.model_size_limit = 0.5
args.bops_limit = 0.5
args.latency_limit = 0.5

In [68]:
Hutchinson_trace = np.array([0.19270050525665214, 0.026702478528024147, 0.09448622912168206, 0.07639402896166123, 0.009688886813818761, 0.006711237132549379, 0.024378638714551495, 0.011769247241317709, 0.00250671175308565, 0.008039989508690044, 0.008477769792080075, 0.0017263438785415808, 0.00313899479806392, 0.004365175962448905, 0.0015507987700405636, 0.0015619333134963605, 0.002701229648662333, 0.002383587881922602, 0.0010320576839146385, 0.0025724433362483905, 0.0046385480090998495, 0.0008540530106982067, 0.003809824585913301, 0.0019535077735770134, 0.0005609235377052988, 0.0018696154002090816, 0.0016933275619534234, 0.000607571622821307, 0.00044498199713291893, 0.0008382285595869651, 0.0006031448720029394, 0.0003951200051233584, 0.000718781026080718, 0.0008380718063555912, 0.00037973490543638077, 0.000828751828521126, 0.0013068615226087446, 0.0004476536996660732, 0.0012669296702365405, 0.0013098433846609064, 0.00032695921254355777, 0.0012137993471687096, 0.0010635341750463423, 0.0002480300900066659, 0.0005943655269220451, 0.0007215500227179646, 0.000612712872680123, 0.00025412911781990733, 0.0005049526807847556, 0.0007025265367691074, 0.0002485941222403296, 0.000525604293216296])
delta_weights_8bit_square = np.array([0.0031, 0.0025, 0.0123, 0.0025, 0.0024, 0.0062, 0.0019, 0.0023, 0.0030, 0.0010, 0.0140, 0.0052, 0.0204, 0.0054, 0.0064, 0.0141, 0.0041, 0.0067, 0.0094, 0.0029, 0.0103, 0.0095, 0.0030, 0.0243, 0.0140, 0.0371, 0.0131, 0.0144, 0.0270, 0.0078, 0.0147, 0.0192, 0.0066, 0.0162, 0.0210, 0.0063, 0.0161, 0.0169, 0.0054, 0.0194, 0.0241, 0.0061, 0.0436, 0.0267, 0.0551, 0.0217, 0.0533, 0.0589, 0.0212, 0.0562, 0.0392, 0.0246])
delta_weights_4bit_square = np.array([0.9257, 0.6624, 3.4605, 0.7805, 0.8040, 2.0657, 0.5947, 0.7596, 0.9826, 0.3381, 4.0922, 1.5506, 6.0288, 1.7186, 2.0701, 4.6080, 1.3128, 2.1613, 3.0710, 0.9209, 3.2794, 3.0883, 0.9841, 7.7154, 4.3872, 11.7131,4.1949, 4.6766, 8.7215, 2.5369, 4.8132, 6.1627, 2.1765, 5.2257, 6.7500, 2.0171, 5.2612, 5.5459, 1.7509, 6.2498, 7.4798, 1.9959, 13.8412,8.3407, 18.0200,6.9997, 15.3605,17.5150,6.4017, 16.0258, 10.7682, 8.0800])
sensitivity_difference_between_4_8 = Hutchinson_trace * ( delta_weights_8bit_square  - delta_weights_4bit_square ) # here is the sensitivity different between 4 and 8
parameters = np.array([0.004096, 0.036864, 0.016384, 0.016384, 0.016384, 0.036864, 0.016384, 0.016384, 0.036864, 0.016384, 0.032768, 0.147456, 0.065536, 0.131072, 0.065536, 0.147456, 0.065536, 0.065536, 0.147456, 0.065536, 0.065536, 0.147456, 0.065536, 0.131072, 0.589824, 0.262144, 0.524288, 0.262144, 0.589824, 0.262144, 0.262144, 0.589824, 0.262144, 0.262144, 0.589824, 0.262144, 0.262144, 0.589824, 0.262144, 0.262144, 0.589824, 0.262144, 0.524288, 2.359296, 1.048576, 2.097152, 1.048576, 2.359296, 1.048576, 1.048576, 2.359296, 1.048576])
bops = np.array([12.845056, 115.605504, 51.380224, 51.380224, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 25.690112, 115.605504, 51.380224, 102.760448, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 25.690112, 115.605504, 51.380224, 102.760448, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224, 25.690112, 115.605504, 51.380224, 102.760448, 51.380224, 115.605504, 51.380224, 51.380224, 115.605504, 51.380224])

latency_int4 = np.array([0.07018191, 0.22481827, 0.27510223, 0.27512618, 0.10512171, 0.22473146, 0.21536969, 0.10506169, 0.22481533, 0.18938694, 0.05091347, 0.19677368, 0.15108796, 0.16824758, 0.0848311, 0.19684267, 0.15120609, 0.08481235, 0.19667935, 0.15121574, 0.08476754, 0.19673882, 0.15123765, 0.04866963, 0.20606097, 0.10150094, 0.169953, 0.09355929, 0.20598819, 0.10146334, 0.09362463, 0.20609737, 0.10188489, 0.09360134, 0.20618122, 0.10150974, 0.09352375, 0.20603792, 0.1015803, 0.09365549, 0.20599906, 0.10143032, 0.08047627, 0.207775, 0.08819472, 0.17924989, 0.10268331, 0.20780669, 0.08795617, 0.10265705, 0.20781191, 0.08793056])
latency_int8 = np.array([0.07079369, 0.36040637, 0.27649506, 0.276513, 0.1611783, 0.36101955, 0.276361, 0.16111273, 0.36064923, 0.24884412, 0.07218692, 0.34899773, 0.19600394, 0.29505887, 0.16074604, 0.34904888, 0.19603683, 0.16067593, 0.34919751, 0.1960506, 0.16089535, 0.34916001, 0.19616036, 0.08172319, 0.33166738, 0.15534599, 0.30712053, 0.15510333, 0.33172639, 0.15541728, 0.15290036, 0.3317953, 0.15551916, 0.15252759, 0.33165544, 0.15529572, 0.15229388, 0.33187364, 0.15545004, 0.15275891, 0.33161443, 0.15566678, 0.08390548, 0.33057286, 0.15200003, 0.33919933, 0.16635967, 0.33092922, 0.15179218, 0.16672966, 0.33086648, 0.1519619])
latency_difference_between_4_8 = latency_int8 - latency_int4


In [69]:
# do some calculatations first
# model size
model_size_32bit = np.sum(parameters) * 4. # MB
model_size_8bit = model_size_32bit / 4.
model_size_4bit = model_size_32bit / 8.
model_size_limit = model_size_4bit + (model_size_8bit - model_size_4bit) * args.model_size_limit

# bops
bops_8bit = bops / 4. / 4.
bops_4bit = bops / 8. / 8.
bops_limit = np.sum(bops_4bit) + (np.sum(bops_8bit) - np.sum(bops_4bit)) * args.bops_limit
bops_difference_between_4_8 = bops_8bit - bops_4bit

# latency 
latency_limit = np.sum(latency_int4) + (np.sum(latency_int8) - np.sum(latency_int4)) * args.latency_limit

In [70]:
# 初始化粒子群
def initialize_particles(num_particles, num_variables):
    particles = []
    velocities = []
    for _ in range(num_particles):
        particle = [random.choice([1, 2]) for _ in range(num_variables)]
        particles.append(particle)
    for _ in range(num_particles):
        velocity = [random.choice([1, 2]) for _ in range(num_variables)]
        velocities.append(velocity)
    return particles, velocities


# 粒子群算法主函数
def PSO(num_particles=30, max_iterations=100, w=0.5, c1=1.5, c2=1.5):
    num_variables = len(parameters)
    particles, velocities = initialize_particles(num_particles, num_variables)
    global_best_particle = particles[0]
    global_best_fitness = fitness_func(global_best_particle)
    local_best_particles = particles[:]  # 初始化每个粒子的局部最优位置为当前位置
    local_best_fitness = [fitness_func(p) for p in particles]  # 记录每个粒子的局部最优适应度
    best_Model_Size_History = []
    best_fitness_History = []

    for iter in range(max_iterations):
        for i, particle in enumerate(particles):
            fitness = fitness_func(particle)
            if fitness < local_best_fitness[i]:  # 更新局部最优
                local_best_particles[i] = particle
                local_best_fitness[i] = fitness
            if fitness < global_best_fitness:  # 更新全局最优
                global_best_fitness = fitness
                global_best_particle = particle
            # 自适应变异
            if np.random.rand() > 0.8:
                index = np.random.randint(0, len(particle))
                if np.random.uniform(1, 2) < 1.5:
                    particle[index] = 1
                else:
                    particle[index] = 2
        for i in range(num_particles):
            particles[i], velocities[i] = update_particle(particles[i], velocities[i], global_best_particle, local_best_particles[i], w, c1, c2)
        global_best_particle = np.array(global_best_particle)
        best_fitness_History.append(global_best_fitness)
        best_Model_Size_History.append(np.sum(global_best_particle * parameters * 4 * 4 / 32))
        # print('{}: {}'.format(iter, global_best_particle))
        
    return global_best_particle, global_best_fitness, best_Model_Size_History, best_fitness_History


# 更新粒子的速度和位置
def update_particle(particle, velocity, global_best_particle, local_best_particle, w, c1, c2):
    new_particle = particle[:]
    new_velocity = velocity[:]
    for i in range(len(particle)):
        r1, r2 = random.random(), random.random()
        # 更新速度
        new_velocity[i] = w * velocity[i] + c1 * r1 * (local_best_particle[i] - particle[i]) + c2 * r2 * (global_best_particle[i] - particle[i])
        # 更新位置
        new_particle[i] = 1 if (new_particle[i] + velocity[i]) < 1.5 else 2

        # new_particle[i] = particle[i] + w * (new_particle[i] - particle[i]) + c1 * r1 * (global_best_particle[i] - particle[i]) + c2 * r2 * (particle[i] - particle[i])
        # new_particle[i] = 1 if new_particle[i] < 1.5 else 2  # 根据速度更新位置，小于1.5取1，大于等于1.5取2

    return new_particle, new_velocity

### Model Size Constrait

In [82]:
# 计算适应度值
def fitness_func(particle):
    if (particle[0] != particle[3]) or (particle[10] != particle[13]) or (particle[23] != particle[26]) or (particle[42] != particle[45]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum(0.5 * particle[i] * parameters[i] for i in range(len(particle)))
        if total <= model_size_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大

# 测试
while(True):
    best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
    model_size = np.sum(best_particle * parameters * 4 * 4 / 32)
    result_4 = (best_particle == 1)
    result_8 = (best_particle == 2)
    Latency = np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8])
    if Latency < 10.81 and model_size < 17.547:
        print("最优粒子：", best_particle)
        print("最优适应度值：", best_fitness)
        print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
        result_4 = (best_particle == 1)
        result_8 = (best_particle == 2)
        print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
        print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))
        break


最优粒子： [2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 1 2 2 1 2 1 1 2
 1 1 2 2 2 2 1 1 2 1 1 2 2 1 2]
最优适应度值： -0.8590237435363499
Model Size: 17.375232
Bops:  181.23571199999998
Latency:  10.624326069999999


### Bops Constrait

In [95]:
# 计算适应度值
def fitness_func(particle):
    if (particle[0] != particle[3]) or (particle[10] != particle[13]) or (particle[23] != particle[26]) or (particle[42] != particle[45]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum([ bops_4bit[i] + (particle[i] - 1) * bops_difference_between_4_8[i] for i in range(len(particle)) ]) 
        if total <= bops_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大


# 测试
while(True):
    best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
    result_4 = (best_particle == 1)
    result_8 = (best_particle == 2)
    size = np.sum(best_particle * parameters * 4 * 4 / 32)
    Bops = np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8])
    if Bops < 145 and size < 16.7 and best_fitness < -0.84:
        print("最优粒子：", best_particle)
        print("最优适应度值：", best_fitness)
        print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
        result_4 = (best_particle == 1)
        result_8 = (best_particle == 2)
        print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
        print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))
        break


最优粒子： [2 2 2 2 2 2 2 2 1 1 2 1 2 2 2 2 2 2 1 2 2 1 2 2 1 2 2 1 1 1 1 1 2 2 1 2 2
 1 1 2 1 1 2 1 2 2 2 1 1 2 1 1]
最优适应度值： -0.8443517051008629
Model Size: 16.185344
Bops:  143.904768
Latency:  9.679219620000001


# Latency Constrait

In [85]:
# 计算适应度值
def fitness_func(particle):
    if (particle[0] != particle[3]) or (particle[10] != particle[13]) or (particle[23] != particle[26]) or (particle[42] != particle[45]):
        return float('inf')  # 违反约束条件，适应度值设为无穷大
    else:
        total = sum([ latency_int4[i] + (particle[i] - 1) * latency_difference_between_4_8[i] for i in range(len(particle)) ])  # if 4 bit, x - 1 = 0, we use bops_4, if 8 bit, x-1=2, we use bops_diff + bos_4 = bops_8
        if total <= latency_limit:
            return sum( [ (particle[i] - 1) * sensitivity_difference_between_4_8[i] for i in range(len(particle)) ] )  
        else:
            return float('inf')  # 违反约束条件，适应度值设为无穷大


# 测试
while(True):
    best_particle, best_fitness, best_Model_Size_History, best_fitness_History = PSO()
    result_4 = (best_particle == 1)
    result_8 = (best_particle == 2)
    Latency = np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8])
    if Latency < 9.76 and best_fitness < -0.85:
        print("最优粒子：", best_particle)
        print("最优适应度值：", best_fitness)
        print('Model Size:', np.sum(best_particle * parameters * 4 * 4 / 32))
        result_4 = (best_particle == 1)
        result_8 = (best_particle == 2)
        print("Bops: ", np.sum(bops_4bit[result_4]) + np.sum(bops_8bit[result_8]))
        print("Latency: ", np.sum(latency_int4[result_4]) + np.sum(latency_int8[result_8]))
        break


最优粒子： [2 2 2 2 2 2 2 2 1 1 2 1 2 2 2 2 2 2 1 2 2 1 2 2 1 2 2 2 1 1 2 1 1 2 1 1 2
 1 1 2 1 1 2 1 2 2 2 1 2 2 1 1]
最优适应度值： -0.8500742296435508
Model Size: 16.709632
Bops:  146.313216
Latency:  9.75645515
